In [1]:
!pip install facenet_pytorch

In [2]:
!pip install matplotlib

In [3]:
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from PIL import Image
from torch import nn

import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torch.utils.data import Dataset
from torchvision import transforms
import glob
import os
# from tensorflow.keras.models import load_model

# from tensorflow.keras.models import Model
import os
import warnings


from torch.utils.data import random_split
from torch.utils.data import DataLoader,Dataset
import torch.nn.functional as F
import torch.optim as optim



/home/csgrad/kaushik3/miniconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        self.gender_mapping = {'male': 0, 'female': 1}
        self.ethnicity_mapping = {'white': 0, 'black': 1, 'asian': 2, 'hispanic': 3}
        self.data['nameNum'] = self.data['name'].astype('category').cat.codes
        self.data['nameNum'] = self.data['nameNum'].astype(int)


    def __len__(self):
        return len(self.data)


    
    def getAgeLabel(self,value1):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        if(class_ranges[0][0]<=value1 and value1<class_ranges[0][1]):
            return 0
        elif(class_ranges[1][0]<=value1 and value1<class_ranges[1][1]):
            return 1
        elif(class_ranges[2][0]<=value1 and value1<class_ranges[2][1]):
            return 2
        elif(class_ranges[3][0]<=value1 and value1<class_ranges[3][1]):
            return 3
        else:
            return 0
   

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        image_path = '/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/Data/CELEBTEST/'+row['name']+'/' + row['filename']  # Assuming images are in a folder named 'images'
        
        try:
            image = Image.open(image_path)
        except Exception as e:
            # Handle the error, for example, you can return a placeholder image
            print("here")
            #self.__getitem__(idx + 1)
            #image = Image.new('RGB', (224, 224))  # Create a blank image
        
        image = Image.open(image_path)
        age = row['age']
        
        if(row['age']<=0):
            age=35
        label = {
            'age': self.getAgeLabel(age),
            'gender': self.gender_mapping.get(row['gender'], 0),  # -1 for unknown
            'ethnicity': self.ethnicity_mapping.get(row['ethnicity'], 0),
            'age1':age,
            'name': row['nameNum']
        
        }
        #print(row['name'])
        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
# csv_file = '/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv'  # Replace with the actual path to your CSV file
# df = pd.read_csv(csv_file)

# # Create a list to store the indices of rows with missing files
# rows_to_remove = []
# count=0
# # Iterate through the DataFrame and check if the files exist
# for index, row in df.iterrows():
#     image_path = '/home/csgrad/byalavar/FHE/celebSet/CELEBTEST/CELEBTEST/'+row['name']+'/' + row['filename'] 
#     if not os.path.exists(image_path):
#         rows_to_remove.append(index)
#         count=count+1
# df = df.drop(rows_to_remove)
# df.to_csv(csv_file, index=False)
# print(count)

In [6]:
transform = transforms.Compose([
    transforms.Resize((112, 112)),  # Resize the image to the desired size
    transforms.ToTensor(),          # Convert the image to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])


In [23]:
# trainSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentity.csv', transform=transform, get_pairs=True, num_pairs=25)
# trainloader = DataLoader(trainSet, batch_size=16, shuffle=True, num_workers = 2)

# testSet = CustomDataset('/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/celebSetIdentityTest.csv', transform=transform, get_pairs=False)
# testloader = DataLoader(testSet, batch_size=16, shuffle=False)

trainSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', transform=transform)
trainloader = DataLoader(trainSet, batch_size=16, shuffle=False, num_workers=2)

testSet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', transform=transform)
testloader = DataLoader(testSet, batch_size=16, shuffle=False, num_workers=2)


In [24]:
def imshow(img):
        img = img / 2 + 0.5     # unnormalize
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)))
        plt.show()

In [25]:
# tempIter = iter(testloader)
# images,gen_pairs, imp_pairs = next(tempIter)
# # imshow(images[0])
# # print(labels['age'][0],labels['gender'][0],labels['ethnicity'][0])
# # print(image_pairs)
# imshow(images[0])
# imshow(gen_pairs[0][0])
# imshow(imp_pairs[0][0])

In [26]:
from collections import namedtuple
import torch
import torch.nn as nn
from torch.nn import Dropout
from torch.nn import MaxPool2d
from torch.nn import Sequential
from torch.nn import Conv2d, Linear
from torch.nn import BatchNorm1d, BatchNorm2d
from torch.nn import ReLU, Sigmoid
from torch.nn import Module
from torch.nn import PReLU
import os

def build_model(model_name='ir_50'):
    if model_name == 'ir_101':
        return IR_101(input_size=(112,112))
    elif model_name == 'ir_50':
        return IR_50(input_size=(112,112))
    elif model_name == 'ir_se_50':
        return IR_SE_50(input_size=(112,112))
    elif model_name == 'ir_34':
        return IR_34(input_size=(112,112))
    elif model_name == 'ir_18':
        return IR_18(input_size=(112,112))
    else:
        raise ValueError('not a correct model name', model_name)

def initialize_weights(modules):
    """ Weight initilize, conv2d and linear is initialized with kaiming_normal
    """
    for m in modules:
        if isinstance(m, nn.Conv2d):
            nn.init.kaiming_normal_(m.weight,
                                    mode='fan_out',
                                    nonlinearity='relu')
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight,
                                    mode='fan_out',
                                    nonlinearity='relu')
            if m.bias is not None:
                m.bias.data.zero_()


class Flatten(Module):
    """ Flat tensor
    """
    def forward(self, input):
        return input.view(input.size(0), -1)


class LinearBlock(Module):
    """ Convolution block without no-linear activation layer
    """
    def __init__(self, in_c, out_c, kernel=(1, 1), stride=(1, 1), padding=(0, 0), groups=1):
        super(LinearBlock, self).__init__()
        self.conv = Conv2d(in_c, out_c, kernel, stride, padding, groups=groups, bias=False)
        self.bn = BatchNorm2d(out_c)

    def forward(self, x):
        x = self.conv(x)
        x = self.bn(x)
        return x


class GNAP(Module):
    """ Global Norm-Aware Pooling block
    """
    def __init__(self, in_c):
        super(GNAP, self).__init__()
        self.bn1 = BatchNorm2d(in_c, affine=False)
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.bn2 = BatchNorm1d(in_c, affine=False)

    def forward(self, x):
        x = self.bn1(x)
        x_norm = torch.norm(x, 2, 1, True)
        x_norm_mean = torch.mean(x_norm)
        weight = x_norm_mean / x_norm
        x = x * weight
        x = self.pool(x)
        x = x.view(x.shape[0], -1)
        feature = self.bn2(x)
        return feature


class GDC(Module):
    """ Global Depthwise Convolution block
    """
    def __init__(self, in_c, embedding_size):
        super(GDC, self).__init__()
        self.conv_6_dw = LinearBlock(in_c, in_c,
                                     groups=in_c,
                                     kernel=(7, 7),
                                     stride=(1, 1),
                                     padding=(0, 0))
        self.conv_6_flatten = Flatten()
        self.linear = Linear(in_c, embedding_size, bias=False)
        self.bn = BatchNorm1d(embedding_size, affine=False)

    def forward(self, x):
        x = self.conv_6_dw(x)
        x = self.conv_6_flatten(x)
        x = self.linear(x)
        x = self.bn(x)
        return x


class SEModule(Module):
    """ SE block
    """
    def __init__(self, channels, reduction):
        super(SEModule, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.fc1 = Conv2d(channels, channels // reduction,
                          kernel_size=1, padding=0, bias=False)

        nn.init.xavier_uniform_(self.fc1.weight.data)

        self.relu = ReLU(inplace=True)
        self.fc2 = Conv2d(channels // reduction, channels,
                          kernel_size=1, padding=0, bias=False)

        self.sigmoid = Sigmoid()

    def forward(self, x):
        module_input = x
        x = self.avg_pool(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.sigmoid(x)

        return module_input * x



class BasicBlockIR(Module):
    """ BasicBlock for IRNet
    """
    def __init__(self, in_channel, depth, stride):
        super(BasicBlockIR, self).__init__()
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, depth, (3, 3), (1, 1), 1, bias=False),
            BatchNorm2d(depth),
            PReLU(depth),
            Conv2d(depth, depth, (3, 3), stride, 1, bias=False),
            BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class BottleneckIR(Module):
    """ BasicBlock with bottleneck for IRNet
    """
    def __init__(self, in_channel, depth, stride):
        super(BottleneckIR, self).__init__()
        reduction_channel = depth // 4
        if in_channel == depth:
            self.shortcut_layer = MaxPool2d(1, stride)
        else:
            self.shortcut_layer = Sequential(
                Conv2d(in_channel, depth, (1, 1), stride, bias=False),
                BatchNorm2d(depth))
        self.res_layer = Sequential(
            BatchNorm2d(in_channel),
            Conv2d(in_channel, reduction_channel, (1, 1), (1, 1), 0, bias=False),
            BatchNorm2d(reduction_channel),
            PReLU(reduction_channel),
            Conv2d(reduction_channel, reduction_channel, (3, 3), (1, 1), 1, bias=False),
            BatchNorm2d(reduction_channel),
            PReLU(reduction_channel),
            Conv2d(reduction_channel, depth, (1, 1), stride, 0, bias=False),
            BatchNorm2d(depth))

    def forward(self, x):
        shortcut = self.shortcut_layer(x)
        res = self.res_layer(x)

        return res + shortcut


class BasicBlockIRSE(BasicBlockIR):
    def __init__(self, in_channel, depth, stride):
        super(BasicBlockIRSE, self).__init__(in_channel, depth, stride)
        self.res_layer.add_module("se_block", SEModule(depth, 16))


class BottleneckIRSE(BottleneckIR):
    def __init__(self, in_channel, depth, stride):
        super(BottleneckIRSE, self).__init__(in_channel, depth, stride)
        self.res_layer.add_module("se_block", SEModule(depth, 16))


class Bottleneck(namedtuple('Block', ['in_channel', 'depth', 'stride'])):
    '''A named tuple describing a ResNet block.'''


def get_block(in_channel, depth, num_units, stride=2):

    return [Bottleneck(in_channel, depth, stride)] +\
           [Bottleneck(depth, depth, 1) for i in range(num_units - 1)]


def get_blocks(num_layers):
    if num_layers == 18:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=2),
            get_block(in_channel=64, depth=128, num_units=2),
            get_block(in_channel=128, depth=256, num_units=2),
            get_block(in_channel=256, depth=512, num_units=2)
        ]
    elif num_layers == 34:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=6),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 50:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=4),
            get_block(in_channel=128, depth=256, num_units=14),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 100:
        blocks = [
            get_block(in_channel=64, depth=64, num_units=3),
            get_block(in_channel=64, depth=128, num_units=13),
            get_block(in_channel=128, depth=256, num_units=30),
            get_block(in_channel=256, depth=512, num_units=3)
        ]
    elif num_layers == 152:
        blocks = [
            get_block(in_channel=64, depth=256, num_units=3),
            get_block(in_channel=256, depth=512, num_units=8),
            get_block(in_channel=512, depth=1024, num_units=36),
            get_block(in_channel=1024, depth=2048, num_units=3)
        ]
    elif num_layers == 200:
        blocks = [
            get_block(in_channel=64, depth=256, num_units=3),
            get_block(in_channel=256, depth=512, num_units=24),
            get_block(in_channel=512, depth=1024, num_units=36),
            get_block(in_channel=1024, depth=2048, num_units=3)
        ]

    return blocks


class Backbone(Module):
    def __init__(self, input_size, num_layers, mode='ir'):
        """ Args:
            input_size: input_size of backbone
            num_layers: num_layers of backbone
            mode: support ir or irse
        """
        super(Backbone, self).__init__()
        assert input_size[0] in [112, 224], \
            "input_size should be [112, 112] or [224, 224]"
        assert num_layers in [18, 34, 50, 100, 152, 200], \
            "num_layers should be 18, 34, 50, 100 or 152"
        assert mode in ['ir', 'ir_se'], \
            "mode should be ir or ir_se"
        self.input_layer = Sequential(Conv2d(3, 64, (3, 3), 1, 1, bias=False),
                                      BatchNorm2d(64), PReLU(64))
        blocks = get_blocks(num_layers)
        if num_layers <= 100:
            if mode == 'ir':
                unit_module = BasicBlockIR
            elif mode == 'ir_se':
                unit_module = BasicBlockIRSE
            output_channel = 512
        else:
            if mode == 'ir':
                unit_module = BottleneckIR
            elif mode == 'ir_se':
                unit_module = BottleneckIRSE
            output_channel = 2048

        if input_size[0] == 112:
            self.output_layer = Sequential(BatchNorm2d(output_channel),
                                        Dropout(0.4), Flatten(),
                                        Linear(output_channel * 7 * 7, 512),
                                        BatchNorm1d(512, affine=False))
        else:
            self.output_layer = Sequential(
                BatchNorm2d(output_channel), Dropout(0.4), Flatten(),
                Linear(output_channel * 14 * 14, 512),
                BatchNorm1d(512, affine=False))

        modules = []
        for block in blocks:
            for bottleneck in block:
                modules.append(
                    unit_module(bottleneck.in_channel, bottleneck.depth,
                                bottleneck.stride))
        self.body = Sequential(*modules)

        initialize_weights(self.modules())


    def forward(self, x):
        
        # current code only supports one extra image
        # it comes with a extra dimension for number of extra image. We will just squeeze it out for now
        x = self.input_layer(x)

        for idx, module in enumerate(self.body):
            x = module(x)

        x = self.output_layer(x)
        norm = torch.norm(x, 2, 1, True)
        output = torch.div(x, norm)

        return output, norm



def IR_18(input_size):
    """ Constructs a ir-18 model.
    """
    model = Backbone(input_size, 18, 'ir')

    return model


def IR_34(input_size):
    """ Constructs a ir-34 model.
    """
    model = Backbone(input_size, 34, 'ir')

    return model


def IR_50(input_size):
    """ Constructs a ir-50 model.
    """
    model = Backbone(input_size, 50, 'ir')

    return model


def IR_101(input_size):
    """ Constructs a ir-101 model.
    """
    model = Backbone(input_size, 100, 'ir')

    return model


def IR_152(input_size):
    """ Constructs a ir-152 model.
    """
    model = Backbone(input_size, 152, 'ir')

    return model


def IR_200(input_size):
    """ Constructs a ir-200 model.
    """
    model = Backbone(input_size, 200, 'ir')

    return model


def IR_SE_50(input_size):
    """ Constructs a ir_se-50 model.
    """
    model = Backbone(input_size, 50, 'ir_se')

    return model


def IR_SE_101(input_size):
    """ Constructs a ir_se-101 model.
    """
    model = Backbone(input_size, 100, 'ir_se')

    return model


def IR_SE_152(input_size):
    """ Constructs a ir_se-152 model.
    """
    model = Backbone(input_size, 152, 'ir_se')

    return model


def IR_SE_200(input_size):
    """ Constructs a ir_se-200 model.
    """
    model = Backbone(input_size, 200, 'ir_se')

    return model


adaface_models = {
    'ir_18':"/home/csgrad/byalavar/FHE/HEAAN/FA_CVPR_Exp/adaface_ir18_webface4m.ckpt",
}


def load_pretrained_model(architecture='ir_18'):
    # load model and pretrained statedict
    assert architecture in adaface_models.keys()
    model = build_model(architecture)
    statedict = torch.load(adaface_models[architecture])['state_dict']
    model_statedict = {key[6:]:val for key, val in statedict.items() if key.startswith('model.')}
    model.load_state_dict(model_statedict)
    model.eval()
    return model

def to_input(pil_rgb_image):
    np_img = np.array(pil_rgb_image)
    brg_img = ((np_img[:,:,::-1] / 255.) - 0.5) / 0.5
    tensor = torch.tensor([brg_img.transpose(2,0,1)]).float()
    return tensor

In [27]:
adaFaceModel = load_pretrained_model('ir_18')

# print(images.shape)
# bgr_image = images[:, [2, 1, 0], :, :]
# print(bgr_image.shape)


#feature, _ = adaFaceModel(bgr_image)

In [28]:
warnings.filterwarnings("ignore", category=DeprecationWarning)  # Ignore DeprecationWarnings
warnings.filterwarnings("ignore", category=FutureWarning) 
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

print('Running on device: {}'.format(device))

Running on device: cuda:1


In [29]:
trainSet.data['age'].min()

-9

In [30]:
def get_normalized_age_value(original_age_value):
    return (original_age_value - trainSet.data['age'].min())/(trainSet.data['age'].max() - trainSet.data['age'].min())


def get_original_age_value(normalized_age_value):
    return normalized_age_value * (trainSet.data['age'].max()  - trainSet.data['age'].min()) + trainSet.data['age'].min()

In [31]:
from torch.nn.modules import MSELoss
from torch.nn.modules.loss import CrossEntropyLoss
from torch.nn.parallel import DataParallel

class GenderDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size,64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        
    def forward(self, x):
        return self.layer(x)

class AgeDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 4)
        )
        
    def forward(self, x):
        return self.layer(x)
    
class EthnicityDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 2)
        )
        
    def forward(self, x):
        return self.layer(x)
    
class IdentityDet(nn.Module):
    def __init__(self, compression_size):
        super().__init__()
        self.layer = nn.Sequential(
            nn.Linear(compression_size, 128), 
            nn.ReLU(), 
            nn.Linear(128, 80))
        
    def forward(self, x):
        return self.layer(x)
    

class Matryoshka_CE_Loss(nn.Module):
    def __init__(self, relative_importance = 1.0):
        super(Matryoshka_CE_Loss, self).__init__()
        self.criterion = nn.CrossEntropyLoss()
        self.relative_importance = relative_importance # usually set to all ones
    
    def forward(self, output, target):
        loss = 0
        for i in range(len(output)):
            # loss += self.relative_importance[i] * self.criterion(output[i], target)
            loss += self.relative_importance * self.criterion(output[i], target)
        return loss

class MRL_Linear_Layer(nn.Module):
    def __init__(self, nesting_list, num_classes=1000, efficient=False, **kwargs):
        super(MRL_Linear_Layer, self).__init__()
        self.nesting_list=nesting_list
        self.num_classes=num_classes # Number of classes for classification
        self.efficient = efficient
        if self.efficient:
            setattr(self, f"nesting_classifier_{0}", nn.Linear(nesting_list[-1], self.num_classes, **kwargs))		
        else:	
            for i, num_feat in enumerate(self.nesting_list):
                setattr(self, f"nesting_classifier_{i}", nn.Linear(num_feat, self.num_classes, **kwargs))	


    def forward(self, x):
        nesting_logits = ()
        for i, num_feat in enumerate(self.nesting_list):
            if self.efficient:
                if self.nesting_classifier_0.bias is None:
                    nesting_logits+= (torch.matmul(x[:, :num_feat], (self.nesting_classifier_0.weight[:, :num_feat]).t()), )
                else:
                    nesting_logits+= (torch.matmul(x[:, :num_feat], (self.nesting_classifier_0.weight[:, :num_feat]).t()) + self.nesting_classifier_0.bias, )
            else:
                nesting_logits +=  (getattr(self, f"nesting_classifier_{i}")(x[:, :num_feat]),)

        return nesting_logits

class MRL_Model(nn.Module):
    def __init__(self, compression_size, efficient=False):
        super(MRL_Model, self).__init__()
        self.ll1 = MRL_Linear_Layer([256], num_classes = compression_size, efficient = efficient)
        self.ll2 = MRL_Linear_Layer([compression_size], num_classes = 80, efficient = efficient)
        # self.flatten = nn.Flatten()
        
    def forward(self, x):
        # x = self.flatten(x)
        x = self.ll1(x)
        return self.ll2(x[0]), x
    

class CompressionLoss(nn.Module):
    def __init__(self, batch_size):
        super(CompressionLoss, self).__init__()
        self.batch_size = batch_size

    def forward(self, img, genuine_imgs, imposter_imgs, genuine_sim, imposter_sim):
        # img -> base image embedding
        # genuine_imgs, imposter_imgs -> Compressed embeddings of dimension 128
        # genuine_sim, imposter_sim -> cosine smilairty scores of uncompressed embeddings of dimension 512


        # Compute cosine similarity loss
        genuine_loss = 0
        for i in range(len(genuine_imgs)):
            sim_score = F.cosine_similarity(img, genuine_imgs[i])
            genuine_loss += torch.mean(torch.abs(sim_score - genuine_sim[i].view(-1)))

        imp_loss = 0
        for i in range(len(imposter_imgs)):
            imp_sim_score = F.cosine_similarity(img, imposter_imgs[i])
            imp_loss += torch.mean(torch.abs(imp_sim_score - imposter_sim[i].view(-1)))

        
        # Compute Covariance Loss
        # matrix -> Concatenates compressed genuine and imposter embeddings
        
        matrix = torch.cat([genuine_imgs[0], imposter_imgs[0]], dim=0)
        for i in range(1, len(genuine_imgs)):
            matrix = torch.cat([matrix, genuine_imgs[i]], dim = 0)
            matrix = torch.cat([matrix, imposter_imgs[i]], dim = 0)


        mean_matrix = torch.mean(matrix, dim = 0)
        mx = torch.matmul(mean_matrix.t(), mean_matrix)
        vx = torch.matmul(matrix.t(), matrix) / self.batch_size # Dividing by batch size as done here https://github.com/human-analysis/hers-encrypted-image-search/blob/master/deep_mds%2B%2B/nntools/tensorflow/networks/deepmds.py#L94
        cov_matrix = mx - vx
        diag = torch.diag(cov_matrix.diag())
        cov_loss = torch.mean(torch.abs(cov_matrix - diag))
        
        # Compute Supervise Loss - Not used for ArcFace
        # # g_dist -> Concatenates compressed genuine embeddings
        # g_dist = torch.cat([genuine_imgs[0], genuine_imgs[1]], dim = 0)
        # for i in range(2, len(genuine_imgs)):
        #     g_dist = torch.cat([g_dist, genuine_imgs[i]], dim = 0)

        # # g_dist -> Concatenates compressed imposter embeddings
        # imp_dist = torch.cat([imposter_imgs[0], imposter_imgs[1]], dim = 0)
        # for i in range(2, len(imposter_imgs)):
        #     imp_dist = torch.cat([imp_dist, imposter_imgs[i]], dim = 0)

        # gloss = (torch.mean(g_dist) + 1.0) * 0.5
        # iloss = (torch.mean(imp_dist) + 1.0) * 0.5

        # lda_term = iloss / gloss
        # return genuine_loss + imp_loss + 10 * cov_loss 
        return 10*genuine_loss + imp_loss + cov_loss


In [32]:
import math
import numpy
def generate_C(C_range, m):
    """ Randomly generates m coefficients for the PolyProtect mapping.

    **Inputs:**

    C_range : integer
        The absolute min/max values of the coefficients range.

    m : int
        The number of coefficients to generate.

    **Outputs:**

    C : 1D numpy array of integers
        Array of m coefficients.

    """

    # Calculate coefficient range (excluding 0):
    neg_range = numpy.arange(-1 * C_range, 0)
    pos_range = numpy.arange(1, C_range + 1)
    whole_range = numpy.concatenate([neg_range, pos_range])

    # Randomly generate m unique coefficients:
    C = numpy.random.permutation(whole_range)[0 : m] # randomly permute the whole range and pick the first few m values

    return C


def generate_E(m):
    """ Randomly generates m exponents for the PolyProtect mapping.

    **Inputs:**

    m : int
        The number of exponents to generate.

    **Outputs:**

    E : 1D numpy array of integers
        Array of m exponents.

    """

    # Randomly generate m unique exponents:
    E = numpy.random.permutation(range(1, m + 1))[0 : m] # permute the integers in the range [1, m]

    return E


def polyprotect(overlap, V):
    """ Maps an embedding to a PolyProtected template.

    **Inputs:**

    overlap : int
        The amount of overlap between sets of embedding elements used to generate each PolyProtected element (0, 1, 2, 3, or 4).

    V : torch.Tensor
        The embedding as a PyTorch tensor.

    **Outputs:**

    P : torch.Tensor
        The PolyProtected template as a PyTorch tensor.

    """
    # print("V = ",V.shape)
    C = torch.tensor([-42, -35, 31, 4], dtype=torch.float32, device=V.device)
    E = torch.tensor([3, 2, 1, 4], dtype=torch.float32, device=V.device)

    if C.shape[0] != E.shape[0]:
        print("Number of coefficients and exponents must be the same.")
        return None
    #print("here")
    # print(C.shape)
    m = C.shape[0] # number of embedding elements used to generate each PolyProtected element
    step_size = m - overlap
    decimal_remainder, integer = math.modf((V.shape[1] - m) / step_size)
    if decimal_remainder > 0:
        padding = math.ceil((1 - decimal_remainder) * step_size)
    else:
        padding = 0
   # print("here1")
    # Pad V by "padding" zeros at the end
    V = torch.cat((V, torch.zeros(padding, device=V.device)), dim=1)

    starting_indices = torch.arange(0, V.shape[1] - m + 1, step_size)
    #print("here2")
    P = torch.zeros((V.shape[0],len(starting_indices)), device=V.device)
    
    for storage_ind, ind in enumerate(starting_indices):
        
        final_ind = ind + m
        crnt_word = V[:, ind:final_ind]
        # print(crnt_word.shape)
        P[:, storage_ind] = torch.sum(C * (crnt_word ** E), dim = 1)
        # print(P)
    #print("here3")
    return P

In [33]:

class faceAnalytics(nn.Module):

    def __init__(self):
        super().__init__()
        
        self.layer1=nn.Linear(128,64)
        self.dropout1=nn.Dropout(0.2)
        self.layer2=nn.Linear(64,32)
        #self.layer3=nn.Linear(1024,512)
        # self.layer4=nn.Linear(128,64)
        self.dropout2=nn.Dropout(0.2)
        self.genderOut = nn.Sequential(
            nn.Linear(128,64),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )
        self.ageOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,4)
        )
        self.ethnicityOut = nn.Sequential(
            nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,2)
        )

        self.identity = nn.Sequential(
            nn.Linear(128,64), 
            nn.ReLU(), 
            nn.Linear(64,80))
        # self.maxVal = 0
        # self.min=0
        
    
    def writeResult(self,result):
       output_directory=""
       file_name = "resultAge.txt"

       with open(os.path.join(output_directory, file_name), "w") as file:
        for value in result:
            file.write(f"{value}\n")
    
    def forward(self,x):

        #print("Input",x[0])
        # x=self.layer1(x)
        # x = nn.ReLU()(x)
        # x=self.layer2(x)
        # x = nn.ReLU()(x)

        gender = self.genderOut(x)
        age = self.ageOut(x)
        ethn = self.ethnicityOut(x)
        id = self.identity(x)
        # identity = self.identity(x)
        return gender, age, ethn, id
    
    
    def are_values_in_same_class(self,value1, value2):

    # Define class ranges
        class_ranges = [(15, 22),(22,40),(40,60),(60,80)]
    
    # Check if both values fall into the same class range
        for class_range in class_ranges:
            if class_range[0] <= value1 < class_range[1] and class_range[0] <= value2 < class_range[1]:
                return True
    
        return False    
    
    def trainMRL(self, compressed_dim, trainloader, adaFace, MRL_Model, MRL_Loss, device, episodes):
        # input_dim = 512
        # compressed_dim = 128
        model = MRL_Model
        # Define optimizer and loss function
        
        optimizer = optim.Adam(model.parameters(), lr=1e-4)
        MRLoss = MRL_Loss.to(device)
        model.train()
        for ep in range(episodes):
            total_loss = 0
            print("Starting " + str(ep))

            for img, data in trainloader:
                img = img.to(device=device)
                label = data['name'].to(device=device)
                inputs = img[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings, _ = adaFace(inputs)
                output, _ = model(embeddings)
                loss = MRLoss(output, label)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            print(f"Epoch [{ep + 1}/{episodes}], Loss: {total_loss}")
        torch.save(model, f'MRL_CelebSet_512x{compressed_dim}_AdaFace_{episodes}.pt')
        print("Training complete!")

    def testMRL(self, trainloader, testloader, adaFace, MRL_Model, idModel, genderModel, ageModel, ethModel, device, episodes):
        # self.train()
        learningRate = 0.001
        gender_loss = nn.CrossEntropyLoss().to(device) 
        age_loss = nn.CrossEntropyLoss().to(device) 
        ethn_loss = nn.CrossEntropyLoss().to(device)
        identityLoss = nn.CrossEntropyLoss().to(device)
        IdOptimizer = torch.optim.Adam(idModel.parameters(), lr=learningRate)
        AgeOptimizer = torch.optim.Adam(ageModel.parameters(), lr=learningRate)
        GenderOptimizer = torch.optim.Adam(genderModel.parameters(), lr=learningRate)
        EthOptimizer = torch.optim.Adam(ethModel.parameters(), lr=learningRate)
        model = MRL_Model
        model.eval()
        idModel.train()
        genderModel.train()
        ageModel.train()
        ethModel.train()
        for ep in range(0, episodes):
            total_gender_loss = 0
            total_age_loss = 0
            total_ethn_loss = 0
            total_id_loss = 0
            for i, data in enumerate(trainloader):

                inputs = data[0].to(device=device)
                age_label = (data[1]["age"]).to(device=device)
                gender_label = data[1]["gender"].to(device=device)
                ethn_label = data[1]["ethnicity"].to(device=device)
                id_label = data[1]["name"].to(device=device)
                inputs = inputs[:, [2, 1, 0], :, :]
                #print(inputs.shape)
                embeddings, _ = adaFace(inputs)
                output, compressedOutput = model(embeddings)
                # print(output[0].shape)
                # print(compressedOutput[0].shape)
                # return
                compressedOutput = polyprotect(3, compressedOutput[0])
                # print(compressedOutput.shape)
                output1 = torch.clone(compressedOutput)
                id = idModel(output1)
                output2 = torch.clone(compressedOutput)
                gender = genderModel(output2)
                output3 = torch.clone(compressedOutput)
                age = ageModel(output3)
                output4 = torch.clone(compressedOutput)
                eth = ethModel(output4)
                # gender, age, ethn, id = self(new_output)

                idLoss = identityLoss(id, id_label) 
                ageLoss = age_loss(age, age_label) 
                genLoss =  gender_loss(gender, gender_label)
                ethLoss = ethn_loss(eth, ethn_label) 
                
                IdOptimizer.zero_grad() 
                AgeOptimizer.zero_grad() 
                GenderOptimizer.zero_grad() 
                EthOptimizer.zero_grad()

                idLoss.backward(retain_graph = True)
                ageLoss.backward(retain_graph = True)
                genLoss.backward(retain_graph = True)
                ethLoss.backward(retain_graph = True) 
                
                IdOptimizer.step() 
                AgeOptimizer.step() 
                GenderOptimizer.step() 
                EthOptimizer.step()
                
                total_gender_loss += genLoss.item()
                total_age_loss += ageLoss.item()
                total_ethn_loss += ethLoss.item()
                total_id_loss += idLoss.item()
            print(f"Epoch [{ep + 1}/{episodes}], Id Loss: {total_id_loss}, Gender Loss: {total_gender_loss}, Age Loss: {total_age_loss}, Eth Loss: {total_ethn_loss} ")
        print("Training done!")

        # self.eval()
        count = 0
        genderAcc = 0
        ageAcc = 0
        ethnAcc = 0
        idAcc = 0
        for i, data in enumerate(testloader):

            inputs = data[0].to(device=device)
            age_label = (data[1]["age"]).to(device=device)
            gender_label = data[1]["gender"].to(device=device)
            ethn_label = data[1]["ethnicity"].to(device=device)
            id_label = data[1]["name"].to(device = device)
            inputs = inputs[:, [2, 1, 0], :, :]
            #print(inputs.shape)
            embeddings, _ = adaFace(inputs)
            output, compressedOutput = model(embeddings)
            compressedOutput = polyprotect(3, compressedOutput[0])
            # gender, age, ethn, id = self(compressedOutput[0])

            output1 = torch.clone(compressedOutput)
            id = idModel(output1)
            output2 = torch.clone(compressedOutput)
            gender = genderModel(output2)
            output3 = torch.clone(compressedOutput)
            age = ageModel(output3)
            output4 = torch.clone(compressedOutput)
            eth = ethModel(output4)

            predictedGender = torch.argmax(gender, dim = 1)
            predictedEthn = torch.argmax(eth, dim = 1)
            predictedAge = torch.argmax(age, dim = 1)
            predictedId = torch.argmax(id, dim = 1)
            for j in range(0, predictedGender.shape[0]):
                count = count + 1
                if(predictedGender[j].item()==gender_label[j].item()):
                    genderAcc = genderAcc + 1
            
                if(predictedEthn[j].item()==ethn_label[j].item()):
                    ethnAcc = ethnAcc + 1

                if(predictedAge[j].item() == age_label[j].item()):
                    ageAcc = ageAcc + 1
                
                if(predictedId[j].item() == id_label[j].item()):
                    idAcc = idAcc + 1

        print("Id Accuracy : ", idAcc / count, "Gender Accuracy : ", genderAcc / count, " Age Acc : ", ageAcc / count, " ethnAcc : ", ethnAcc/count)
    
    
   

In [34]:
torch.cuda.device_count()

4

In [35]:
batch_size = 16
FAmodel=faceAnalytics()
# nesting_list = [64, 128, 256]
# num_classes = 80
compression_size = 64
# MRL_model = MRL_Model(compression_size)
MRL_model = torch.load('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/FGPaper/Code/MRL/MRL_CelebSet_512x64_AdaFace_20.pt')
MRL_Loss = Matryoshka_CE_Loss()
FAmodel.to(device)
MRL_model.to(device)
adaFaceModel.to(device)
adaFaceModel.eval()

Backbone(
  (input_layer): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): PReLU(num_parameters=64)
  )
  (output_layer): Sequential(
    (0): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): Dropout(p=0.4, inplace=False)
    (2): Flatten()
    (3): Linear(in_features=25088, out_features=512, bias=True)
    (4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
  )
  (body): Sequential(
    (0): BasicBlockIR(
      (shortcut_layer): MaxPool2d(kernel_size=1, stride=2, padding=0, dilation=1, ceil_mode=False)
      (res_layer): Sequential(
        (0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (2): BatchNorm2d(64, eps=1e-05, momentum=0.1

In [36]:
# FAmodel.trainMRL(compression_size, trainloader, adaFaceModel, MRL_model, MRL_Loss, device, 20)

In [21]:
m = 4
# overlap = 3
polyProtectSize = compression_size - m + 1
idModel = IdentityDet(polyProtectSize)
idModel.to(device)
ageModel = AgeDet(polyProtectSize)
ageModel.to(device)
genderModel = GenderDet(polyProtectSize)
genderModel.to(device)
ethModel = EthnicityDet(polyProtectSize)
ethModel.to(device)
FAmodel.testMRL(trainloader, testloader, adaFaceModel, MRL_model, idModel, genderModel, ageModel, ethModel, device, 5)

Epoch [1/5], Id Loss: 4228.504455263726, Gender Loss: 767.5094219833845, Age Loss: 2402.765411056578, Eth Loss: 365.16578882593603 
Epoch [2/5], Id Loss: 1893.3193493662257, Gender Loss: 509.0271213449887, Age Loss: 1712.691794604063, Eth Loss: 213.1162373920979 
Epoch [3/5], Id Loss: 1429.3192988506853, Gender Loss: 449.7805025935304, Age Loss: 1413.558504840359, Eth Loss: 163.47917260463691 
Epoch [4/5], Id Loss: 1185.1744020236274, Gender Loss: 419.4918862645427, Age Loss: 1190.1566150896251, Eth Loss: 136.09744303844462 
Epoch [5/5], Id Loss: 1061.8027829648954, Gender Loss: 401.0982089775425, Age Loss: 1040.438982438296, Eth Loss: 125.57719609562919 
Training done!
Id Accuracy :  0.93625 Gender Accuracy :  0.968125  Age Acc :  0.89125  ethnAcc :  0.9825


In [21]:
compressed_outputs_list = []
for i, data in enumerate(trainloader):
    inputs = data[0].to(device=device)
    inputs = inputs[:, [2, 1, 0], :, :]
    #print(inputs.shape)
    embeddings, _ = adaFaceModel(inputs)
    output, compressedOutput = MRL_model(embeddings)
    compressedOutput = polyprotect(3, compressedOutput[0])
    compressed_output_np = compressedOutput.detach().cpu().numpy()
    compressed_outputs_list.append(compressed_output_np)

for i, data in enumerate(testloader):
    inputs = data[0].to(device=device)
    inputs = inputs[:, [2, 1, 0], :, :]
    #print(inputs.shape)
    embeddings, _ = adaFaceModel(inputs)
    output, compressedOutput = MRL_model(embeddings)
    compressedOutput = polyprotect(3, compressedOutput[0])
    compressed_output_np = compressedOutput.detach().cpu().numpy()
    compressed_outputs_list.append(compressed_output_np)


combined_compressed_output = np.concatenate(compressed_outputs_list, axis=0)

file_path_combined = 'MRL+PP_AdaFace_Celebset_output.txt'

# Write the combined array to a text file with floating-point format
np.savetxt(file_path_combined, combined_compressed_output, fmt='%.6f')

In [37]:
# compressed_outputs_list = []
# for i, data in enumerate(trainloader):
#     inputs = data[0].to(device=device)
#     inputs = inputs[:, [2, 1, 0], :, :]
#     #print(inputs.shape)
#     embeddings, _ = adaFaceModel(inputs)
#     # output, compressedOutput = MRL_model(embeddings)
#     compressedOutput = polyprotect(3, embeddings)
#     compressed_output_np = compressedOutput.detach().cpu().numpy()
#     compressed_outputs_list.append(compressed_output_np)

# for i, data in enumerate(testloader):
#     inputs = data[0].to(device=device)
#     inputs = inputs[:, [2, 1, 0], :, :]
#     #print(inputs.shape)
#     embeddings, _ = adaFaceModel(inputs)
#     # output, compressedOutput = MRL_model(embeddings)
#     compressedOutput = polyprotect(3, embeddings)
#     compressed_output_np = compressedOutput.detach().cpu().numpy()
#     compressed_outputs_list.append(compressed_output_np)


# combined_compressed_output = np.concatenate(compressed_outputs_list, axis=0)

# file_path_combined = 'PP_AdaFace_Celebset_output.txt'

# # Write the combined array to a text file with floating-point format
# np.savetxt(file_path_combined, combined_compressed_output, fmt='%.6f')

#Ignore Below

In [20]:
warnings.filterwarnings("ignore", category=UserWarning)


processedTensor = torch.zeros((len(testSet), 128))
identityLabel = torch.zeros((len(testSet)))
count = 0

for i,data in enumerate(testloader):
    
    inputs = data[0].to(device=device)
    iden_label = data[1]['name'].to(device=device)
    inputs = inputs[:, [2, 1, 0], :, :]    
    embeddings,_ = adaFaceModel(inputs)
    encoded, _ = aeModel(embeddings)
    # compressed_img1 = compressionModel1(embeddings)
    # compressed_img2 = compressionModel2(compressed_img1)
    # compressed = combinedModel(embeddings)
    processedTensor[count : count + embeddings.shape[0]] = torch.tensor(encoded)
    identityLabel[count:count + iden_label.shape[0]] = torch.tensor(iden_label)


    count = count + embeddings.shape[0]
    

In [21]:
identityAccuracy = 0

count = 0

while(count < processedTensor.shape[0]):

    currentFace = processedTensor[count : count + 1]
    cosine_similarity = nn.functional.cosine_similarity(currentFace, processedTensor)
    if(identityLabel[torch.topk(cosine_similarity, k = 2)[1][1]].item() == identityLabel[count].item()):
        identityAccuracy = identityAccuracy +1
    count = count + 1
    # print(count)
print(identityAccuracy/count)

0.8325


In [ ]:
# FAmodel.trainCompressionModel(trainloader,testloader,adaFaceModel,compressionModel, compressionLoss, device, 10)

In [ ]:
trainSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/celebSET_final_v1.csv', get_pairs=False, transform=transform)
trainloaderDet = DataLoader(trainSetDet, batch_size=16, shuffle=True, num_workers=2)

testSetDet = CustomDataset('/home/csgrad/byalavar/FHE/celebSet/final_clebSET_test.csv', get_pairs=False, transform=transform)
testloaderDet = DataLoader(testSetDet, batch_size=16, shuffle=False, num_workers=2)


In [31]:
aeModel = torch.load('/home/csgrad/kaushik3/PolyProtect/PolyProtect_HEAAN/HEAAN/code/CompressionCode/AeModel_20_bs32_82acc_best.pt')
aeModel.to(device)
FAmodel=faceAnalytics()
FAmodel.to(device)
aeModel.eval()
trainingAcc = FAmodel.trainModel(trainloader,testloader,adaFaceModel,aeModel, device,5)

Gender Accuracy: 0.9623919084766325 Age Acc: 0.6096150319513606  ethnAcc: 0.9649875156935491
Test Gender Accuracy: 0.934375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.925625
Gender Accuracy: 0.969106631494308 Age Acc: 0.5989786849863872  ethnAcc: 0.9704608613466124
Gender Accuracy: 0.971490640296802 Age Acc: 0.5989786849863872  ethnAcc: 0.97325396041699
Test Gender Accuracy: 0.954375 Test Age Accuracy: 0.571875 Test ethnicity Accuracy: 0.921875
Gender Accuracy: 0.9732398538560284 Age Acc: 0.5989786849863872  ethnAcc: 0.9745235509035253


KeyboardInterrupt: 

In [ ]:
FAmodel.test(testloaderDet,adaFaceModel, compressionModel, device)

In [ ]:
torch.save(model,"arcFaceCelebSetBase.pt")

In [ ]:
torch.save(model,"negGenderBaseWiki.pt")

In [ ]:
plt.plot(trainingAcc)

In [ ]:
torch.save(model,"model92.pt")

In [ ]:
count=0
input1=[]
for i,data in enumerate(val_dataloader):
    
    if(count==0):
     inputs=resnet(data["image"].to(device))

 
     input1 = polyprotect(0,inputs[0])

     break
    count=count+1

In [ ]:
input1

In [ ]:
for param in model.parameters():
    print(param.shape)
    print(param[0])
    print(torch.dot(input1,param[1]))
    break 

In [ ]:
input1

In [ ]:
output_directory=""
file_name = "input1.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in input1:
            file.write(f"{value}\n")

In [ ]:
model=faceAnalytics()
model=torch.load("/home/csgrad/byalavar/FHE/HEAAN/modelUsing0.pt")
model.to(device)
model.test(dataloader,device)

In [ ]:
torch.save(model,"modelUsing0.pt")

In [ ]:
count=0
for param in model.parameters():
    print(param.shape)
    count=count+1
    if(count==2):
        print(param)

In [ ]:
a=nn.Linear(4,2)
input1=torch.rand((1,4))
print("input1",input1)
for param in a.parameters():
    print("param",param)
print(a(input1))


In [ ]:
model=faceAnalytics()
model=torch.load("modelUsing0.pt")
model.to(device)
count=0
ageBias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==7):
       ageBias = param.tolist()
    count=count+1
print(len(ageBias),len(ageBias[0]))


In [ ]:
ageBias

In [ ]:
bias2

In [ ]:
# Create a list of lists with shape (512, 128)


# Define the directory where you want to save the text files
output_directory = "ageWeights"

# Create the output directory if it doesn't exist
import os
os.makedirs(output_directory, exist_ok=True)

# Write each list to a separate text file


# Write each list to a separate text file
for i, sublist in enumerate(ageWeights):
    # Define the file name with leading zeros
    file_name = f"{i:03d}.txt"

    # Write each value in the sublist on a new line
    with open(os.path.join(output_directory, file_name), "w") as file:
        for value in sublist:
            file.write(f"{value}\n")

print("Files saved successfully.")


In [ ]:
count=1
layer1Bias=[]
for param in model.parameters(): 
    print(param.shape)
    if(count==2):
       layer1Bias = param.tolist()
       break
    count=count+1
print(len(layer1Bias),len(layer1Bias[0]))

In [ ]:
output_directory=""
file_name = "ageBias.txt"

with open(os.path.join(output_directory, file_name), "w") as file:
        for value in ageBias:
            file.write(f"{value}\n")